In [4]:
import torch 
from torch import nn
import torch.nn.functional as F

import math

In [10]:
nn.ModuleDict(dict(
    wte = nn.Embedding(20, 50),
    wpe = nn.Embedding(2, 50), 
    drop = nn.Dropout(0.2),
    ))

ModuleDict(
  (wte): Embedding(20, 50)
  (wpe): Embedding(2, 50)
  (drop): Dropout(p=0.2, inplace=False)
)

# Attention workflow
1. Input 
2. Word embeddings
3. Positional embeddings
4. Concat(Word, Positional)
5. Normalization (optional)
6. Attention


In [80]:
import torch

# Example dimensions
B = 32  # Batch size
T = 1024  # Sequence length or context window
C = 128 # Embedding dimension

# Generate tokenized input
x = torch.randint(0, 50276, (50276,))
print(x.shape)
# Ensure the length of x is divisible by T
num_tokens = B * T
x = x[:num_tokens]  # Truncate or pad x as necessary

print(x.shape)
# Reshape to (B, T)
x = x.view(B, T)

x.shape


torch.Size([50276])
torch.Size([1024])


torch.Size([1, 1024])

In [97]:
# Example dimensions
B = 32  # Batch size
T = 50  # Desired sequence length
C = 128 # Embedding dimension

# Generate a long tokenized input sequence
x = torch.randint(0, 50276, (50276,))  # Example long sequence

# Chunk the sequence into non-overlapping segments of length T
chunks = [x[i:i + T] for i in range(0, len(x), T)]

# Ensure each chunk is of length T (pad if necessary)
def pad_sequence(seq, length):
    if len(seq) < length:
        return torch.cat([seq, torch.zeros(length - len(seq), dtype=torch.long)])
    return seq

chunks = [pad_sequence(chunk, T) for chunk in chunks]

# Create a batch from the chunks
batches = [chunks[i:i + B] for i in range(0, len(chunks), B)]
len(batches[0])

32

In [98]:
#token embeddings
def txt_emb(vocab_size=50276, n_emb=768):
    wte = nn.Embedding(vocab_size, n_emb)
    return wte

#positional embeddings
def pos_emb(block_size=1024, n_embd=768):
    wpe = nn.Embedding(block_size, n_embd)
    return wpe

T = txt_emb()(x)
b, t = x.size()
pos = torch.arange(0, t)
P = pos_emb()(pos)

ValueError: not enough values to unpack (expected 2, got 1)

In [99]:
#https://arxiv.org/pdf/2406.17763